In [17]:
from pyiem.util import get_dbconn, get_sqlalchemy_conn
import geopandas as gpd

with get_sqlalchemy_conn("postgis") as conn:
    df = gpd.read_postgis("""
WITH obs as (
 SELECT segid, sum(lead - valid) from
 (SELECT segid, valid,  lead(valid) OVER (PARTITION by segid ORDER by valid ASC),
 cond_code from roads_log WHERE valid > '2021-11-01' and valid < '2022-04-04') as foo
 WHERE cond_code > 0 GROUP by segid
)
 SELECT obs.sum, base.simple_geom from obs JOIN
 roads_base base on (obs.segid = base.segid) 

""",
                         conn,
                          geom_col="simple_geom",
                         )

print(len(df.index))

974


In [18]:
import datetime

sts = datetime.datetime(2021,11,1)
ets = datetime.datetime(2022,4,4)
period = (ets - sts).total_seconds()

In [19]:
from pyiem.plot import MapPlot, whitebluegreenyellowred
import numpy as np
import matplotlib.colors as mpcolors
import cartopy.crs as ccrs

cmap = whitebluegreenyellowred()
clevs = np.arange(0,9,1)
norm = mpcolors.BoundaryNorm(clevs, cmap.N)
df["color"] = [x for x in cmap(norm(df["sum"].dt.total_seconds() / period * 100.))]

mp = MapPlot(title='1 Nov 2021 - 4 Apr 2022 Percent Time with Non-Seasonable Driving Condition',
            subtitle='Based on Iowa DOT Road Condition Reports, till 10 PM 4 Apr 2022',
            axisbg='white',
            twitter=True)

df.to_crs(mp.panels[0].crs).plot(
    aspect=None,
    ax=mp.panels[0].ax,
    color=df["color"],
    lw=3,
    zorder=10,
)

mp.draw_colorbar(clevs, cmap, norm, units='percent')
mp.drawcities()
        
mp.postprocess(filename='220405.png')
mp.close()